# Auto Polling
    - The original persona generation worked, but the personas were too generic and didnt give a model context to 
        make accurate predictions of the beliefs of a person
    - Now making a new pipeline that will create personas in different stages to 
        

In [1]:
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import string
import random
import numpy as np
import pickle

In [2]:
def create_directory_if_not_exists(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Load First and Last Names

In [3]:
eths = ["White", "American Indian or Alaska Native", "Black or African American", "Asian", "Hispanic or Latino"]
genders = ['Men', 'Women']

dems = []
for e in eths:
    for g in genders:
        s = f"{e} {g}"
        dems.append(s)
        
first_names = {}
for d in dems:
    # Open the file in read binary mode
    with open(f"Names/First_Names/{d}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        first_names[d] = my_list

last_names = {}
for e in eths:
    # Open the file in read binary mode
    with open(f"Names/Last_Names/{e}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        last_names[e] = my_list

# Distributing Statistics

In [4]:
def distribute_geographic_location():
    random_number = random.random()
    
    if random_number < 0.44:
        return "Rural Montana, United States"
    else:
        return "Urban Montana, United States"
    
def distribute_gender():
    random_number = random.random()
    
    if random_number < 0.49:
        return "Female"
    else:
        return "Male"
    
def distribute_ethnicity():
    random_number = random.random()
    
    if random_number < 0.88:
        return "White"
    elif random_number < 0.94:
        return "American Indian or Alaska Native"
    elif random_number < 0.95:
        return "Black or African American"
    elif random_number < 0.96:
        return "Asian"
    else:
        return "Hispanic or Latino"
    
def distribute_age_and_income():
    random_number = random.random()
    
    if random_number < 0.13:
        age = random.randint(18, 25)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 32413  # Mean of the distribution
        std_dev = 10000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    elif random_number < 0.44:
        age = random.randint(25, 44)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 69192  # Mean of the distribution
        std_dev = 20000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    elif random_number < 0.75:
        age = random.randint(45, 64)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 71064  # Mean of the distribution
        std_dev = 25000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    else:
        age = random.randint(65, 87)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 47015  # Mean of the distribution
        std_dev = 15000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    
def distribute_education_level():
    random_number = random.random()
    
    if random_number < 0.60:
        return "High school graduate"
    elif random_number < 0.94:
        return "Bachelor's Degree or Higher"
    else:
        return "Less than a high school education"
    
def distribute_party():
    """
    # Distribute Politcal Affiliation
    - Reps that say they are conservative at Con
    - Reps say moderate, right leaning
    - Anyone that contradicts or says dont know is moderate
    - Dems that say moderate, left leaning
    - Dems that say liberal, liberal
    """
    random_number = random.random()
    
    if random_number < 0.42:
        return "Conservative"
    elif random_number < 0.56:
        return "Right Leaning"
    elif random_number < 0.66:
        return "Moderate"
    elif random_number < 0.81:
        return "Left Leaning"
    else:
        return "Liberal"
    
def distribute_veteran():
    random_number = random.random()
    
    if random_number < 0.9:
        return "Not a Veteran"
    else:
        return "Is a Veteran"

# Generate Names

In [5]:
def generate_name(eth, gend):
    if gend == 'Male':
        gen = 'Men'
    else:
        gen = 'Women'
    f_names = first_names[f"{eth} {gen}"]
    
    first = f_names[random.randint(0,len(f_names)-1)].strip()
    
    l_names = last_names[f"{eth}"]
    last = l_names[random.randint(0,len(l_names)-1)].strip()
    
    return f"{first} {last}"

In [6]:
def make_persona():
    p = {}
    
    # Distribute Ethnicity
    eth = distribute_ethnicity()
    # Distribute Gender
    gend = distribute_gender()
    
    p['Name'] = generate_name(eth, gend)
    
    # Distribute Age and Income
    age, income = distribute_age_and_income()
    p['Age'] = str(age)
    p['Ethnicity'] = eth
    p['Gender'] = gend
    
    p['Income'] = str(income)

    # Distribute Education Level
    p['Education Level'] = distribute_education_level()
    
    # Distribute Political Affiliation
    p['Political Affiliation'] = distribute_party()
    
    # Distribute Geographic Location
    p['Geographic Location'] = distribute_geographic_location()
    
    # Distribute Veteran Status
    p['Veteran Status'] = distribute_veteran()
    
    return p

In [7]:
poll_size = 343
pp = []
for i in range(poll_size):
    pp.append(make_persona())

In [8]:
TOKENS = 0

In [9]:
def gen_person_occupation(p):
    occ_gen_role = GPT.open_file("Prompts/OccupationGen/occ_gen_role.txt")
    prompt = GPT.open_file("Prompts/OccupationGen/occ_gen_prompt.txt").replace('<<PERSON>>', str(p))
    occ, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.5, tokens = 50, role=occ_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Occupation'] = occ
    
    return p

In [10]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_occ = list(executor.map(gen_person_occupation, pp))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 00423ca20fbe8d146f72adf0a11ed01a in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a094b974ee6d7f02efeb3a3144c97801 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d790ea3695c973f3fea629ab08befcf9 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.

In [11]:
def gen_person_media(p):
    media_gen_role = GPT.open_file("Prompts/MediaGen/media_gen_role.txt")
    prompt = GPT.open_file("Prompts/MediaGen/media_gen_in.txt").replace('<<PERSON>>', str(p))
    media, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.5, tokens = 400, role=media_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Media Consumption'] = media
    
    return p

In [12]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_media = list(executor.map(gen_person_media, pp_occ))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e5c61d97467e93fb226ce17acd2b4b1d in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 88c470018918fc8925ba3ff669ebb915 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e6c190fe9252a7f1cd6882f25b8ffc7c in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.

In [13]:
def gen_person_voting_history(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_voting_history.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.5, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Voting History'] = vh
    
    return p

In [14]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_vote = list(executor.map(gen_person_voting_history, pp_media))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 8105e654675bc64348c43d52bf6117ea in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b3a1b93e9eb929d350568a12df1c6861 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 672d2f65903c3eb3e2c9e034564bdc10 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.

In [15]:
def gen_person_group_membership(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_group_membership.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.5, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Group Membership'] = vh
    
    return p

In [16]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_mem = list(executor.map(gen_person_group_membership, pp_vote))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 20cd7dabf72fa9c0f6590c049b2dd039 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b091f62304edfca55978aab90fa9a5f5 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 8eaf0fc64169afa9ebba01680d8ee14f in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.

In [17]:
def gen_person_if(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_influential_figures.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.5, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Influential Figures'] = vh
    
    return p

In [18]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_if = list(executor.map(gen_person_if, pp_mem))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a21ef49305baca6be99f9a19b2d6e336 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID c41e8735b9de3a1154bbf6d614197357 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID ffd5e0143352dee0dc896b061f4de9dd in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.

In [19]:
def gen_person_personal_anecdotes(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/personal_anecdotes.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.5, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Personal Anecdotes'] = vh
    
    return p

In [20]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_anec = list(executor.map(gen_person_personal_anecdotes, pp_if))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7142d2bf6d3f94fcbe34ef8b2374a0a4 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID b55a8ea9f0a99924bd28c40dfb4308d4 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1814003e887e2852727812d61cdaf08d in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.

In [21]:
def gen_person_future(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_future_aspirations_and_concerns.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.5, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Future Aspirations/Concerns'] = vh
    
    return p

In [22]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_future = list(executor.map(gen_person_future, pp_anec))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e2e5576b67d2eb1bf07809572683fc61 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1838cffb5d05e96736415d4a1536ca30 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 3c393fc1d25a3cd7da60dbb31c7ea925 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.

In [23]:
TOKENS

1485993

In [24]:
cost = TOKENS/1000*0.002
print(cost)

2.971986


In [25]:
pp_future[3]

{'Name': 'Paloma Galvan',
 'Age': '46',
 'Ethnicity': 'Hispanic or Latino',
 'Gender': 'Female',
 'Income': '110300',
 'Education Level': "Bachelor's Degree or Higher",
 'Political Affiliation': 'Moderate',
 'Geographic Location': 'Rural Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Occupation': 'Environmental Engineer.',
 'Media Consumption': "Based on the demographic information provided, Paloma Galvan, a 46-year-old Hispanic or Latino female with a Bachelor's Degree or higher and a moderate political affiliation living in rural Montana with an income of $110,300 and working as an environmental engineer, is likely to trust and consume the following media outlets regularly:\n\n1. CNN - Paloma is likely to consume CNN for national and international news coverage, as it is a well-established and reputable news outlet that aligns with her moderate political views.\n\n2. NPR - Paloma is likely to consume NPR for its in-depth and informative reporting on a variety of topic

In [27]:
import json
file_path = "new_personas_343_2.json"

# Open the file in write mode and save the dictionary as JSON
with open(file_path, "w") as json_file:
    json.dump(pp_future, json_file)

In [28]:
for p in pp_future:
    print(p)
    print()

{'Name': 'Cassandra Taylor', 'Age': '31', 'Ethnicity': 'White', 'Gender': 'Female', 'Income': '53300', 'Education Level': 'High school graduate', 'Political Affiliation': 'Conservative', 'Geographic Location': 'Rural Montana, United States', 'Veteran Status': 'Is a Veteran', 'Occupation': 'Veterinary Assistant.', 'Media Consumption': 'Based on the demographics provided, Cassandra Taylor, a 31-year-old white female living in rural Montana, United States, with a conservative political affiliation, high school education level, and an annual income of $53,300, is likely to trust and consume media outlets that align with her political beliefs and geographic location.\n\nAs a conservative, Cassandra is likely to trust and consume media outlets such as Fox News, Newsmax, and One America News Network (OANN). She may also consume conservative talk radio shows such as Rush Limbaugh and Sean Hannity.\n\nLiving in rural Montana, Cassandra may also consume local news outlets such as the Billings Ga

In [28]:
def dict_to_paragraph(persona, residence):
    priorities = ', '.join(persona["Top Priorities"])
    try:
        pain_points = ', '.join(persona["Pain points"])
    except:
        pain_points = ', '.join(persona["Pain Points"])
    
    return (f"Your name is {persona['Name']}, you are a {persona['Age']} year old {persona['Ethnicity']} {persona['Gender']}. "
            f"You live in {residence} and your annual income is ${persona['Income']}. "
            f"Your marital status is {persona['Marital Status']}, and your highest level of education is {persona['Education Level']}. "
            f"You work as a {persona['Occupation']}. You are described as '{persona['Description']}'. "
            f"Your top priorities are {priorities}. You are facing some challenges, including {pain_points}.")
